In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = '/content/drive/MyDrive/BMU/Project 3/train'
validation_dir = '/content/drive/MyDrive/BMU/Project 3/validation'
os.makedirs(validation_dir, exist_ok=True)

train_datagen = ImageDataGenerator(
    rescale=1./255, #normalize_pixels
    validation_split=0.2  #split into 80 train and 20 test
)

target_size = (224,224)
batch_size = 32

Train_df = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    subset='validation'
)

Found 1736 images belonging to 6 classes.
Found 431 images belonging to 6 classes.


In [ ]:
model = Sequential([
    Conv2D(96, 11, strides = (4, 4), padding = 'valid', input_shape=(224, 224, 3), activation = 'relu'),
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'),
    BatchNormalization(),
    Conv2D(256, 11, strides = (1, 1), padding='valid', activation = 'relu'),
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding='valid'),
    BatchNormalization(),
    Conv2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'),
    BatchNormalization(),
    Conv2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'),
    BatchNormalization(),
    Conv2D(256, 3, strides=(1,1), padding='valid', activation = 'relu'),
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'),
    BatchNormalization(),
    Flatten(),
    Dense(units = 4096, activation = 'relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(units = 4096, activation = 'relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(units = 1000, activation = 'relu'),
    Dropout(0.2),
    BatchNormalization(),
    Dense(units = 6, activation = 'softmax')
])

In [ ]:
model.compile(optimizer = SGD(lr=0.001, momentum=0.9),
              loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_num = Train_df.samples
valid_num = validation_generator.samples

In [ ]:
history = model.fit(Train_df,
                    steps_per_epoch=train_num//batch_size,
                    validation_data=validation_generator,
                    epochs=25,
                    validation_steps=valid_num//batch_size)

Epoch 1/25
54/54 [==============================] - 586s 11s/step - loss: 1.8466 - accuracy: 0.4701 - val_loss: 759.4459 - val_accuracy: 0.1707
Epoch 2/25
54/54 [==============================] - 364s 7s/step - loss: 1.2688 - accuracy: 0.6156 - val_loss: 156.0520 - val_accuracy: 0.1707
Epoch 3/25
54/54 [==============================] - 352s 7s/step - loss: 1.1523 - accuracy: 0.6555 - val_loss: 23.9314 - val_accuracy: 0.2837
Epoch 4/25
54/54 [==============================] - 352s 6s/step - loss: 1.0301 - accuracy: 0.6919 - val_loss: 8.9482 - val_accuracy: 0.4712
Epoch 5/25
54/54 [==============================] - 347s 6s/step - loss: 0.9495 - accuracy: 0.7142 - val_loss: 7.8518 - val_accuracy: 0.4615
Epoch 6/25
54/54 [==============================] - 343s 6s/step - loss: 0.8998 - accuracy: 0.7236 - val_loss: 2.4023 - val_accuracy: 0.7067
Epoch 7/25
54/54 [==============================] - 328s 6s/step - loss: 0.7331 - accuracy: 0.7647 - val_loss: 3.3188 - val_accuracy: 0.3726
Epoch 8

In [ ]:
import seaborn as sns
sns.set()

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)